# KEO data example
Created by Sophie Chu, NOAA PMEL, modified by Nancy Williams, USF CMS for use with Ocean Biogeochemical Dynamics Lab course
- Downloading data via url
- Creating a dataframe using pandas
- Calculating monthly means and standard deviations
- Plotting using matplotlib
- Decomposing pCO2 seasonality into thermal and nonthermal components

In [ ]:
# import libraries needed for this analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
# function for data download via url

def download_url(url):
  print("downloading: ",url)
  # assumes that the last segment after the / represents the file name
  # if url is abc/xyz/file.txt, the file name will be file.txt
  file_name_start_pos = url.rfind("/") + 1
  file_name = url[file_name_start_pos:]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    with open(file_name, 'wb') as f:
      for data in r:
        f.write(data)

In [ ]:
# uses above function to download data via url and save file

download_url("https://www.pmel.noaa.gov/co2/timeseries/KEO.txt")

In [ ]:
# read text file into dataframe
keo = pd.read_table('KEO.txt', delim_whitespace=True, comment ='#', header=0,
                       names=('date','time', 'SST', 'SSS', 'pCO2_sw', 'pCO2_air', 'xCO2_air', 'pH_sw'))

In [ ]:
# take a quick look at the beginning and end of the dataframe
keo.head

In [ ]:
keo

In [ ]:
# function to define data time as index and add depth and month columns

def surfdf (_df, date, time, dpth):
    _df['datetime64_ns'] = pd.to_datetime(_df.date+ ' ' + _df.time) # combine date and time into one column
    _df.index = _df.datetime64_ns # set datetime column as index
    _df['Depth'] = dpth # add depth of surface measurements as column
    _df['month'] = _df['datetime64_ns'].dt.month.values #make month column
    
    return _df

# apply function to dataframe

df_keo = surfdf(keo, keo.date, keo.time, 0.5)

In [ ]:
# take a quick look at the beginning and end of the new dataframe

df_keo.head

In [ ]:
df_keo

In [ ]:
# take a quick look at the different data types of variables in the data frame
df_keo.dtypes

In [ ]:
# quick plot
df_keo.pCO2_sw.plot()

In [ ]:
# plotting SST and SSS on one plot

fig, ax = plt.subplots()

im = ax.plot(df_keo.index, df_keo.SST)
im2 = ax.plot(df_keo.index,df_keo.SSS)

ax.set_ylabel('SST and SSS') # fontsize=fs)

plt.savefig('keo_ts.png', bbox_inches='tight', dpi=600) # saves plot as image file
plt.show() # displays plot

In [ ]:
# get average monthly values for dataset
df_keo_avgmonth = df_keo.groupby('month').agg(['mean', 'std', 'count'])
df_keo_avgmonth # displays the results

In [ ]:
df_keo_avgmonth.info()

In [ ]:
# find max monthly mean
df_keo_avgmonth.pCO2_sw['mean'].max()

In [ ]:
# plot monthly average SST
fig, ax = plt.subplots()
im = ax.errorbar(df_keo_avgmonth.index, df_keo_avgmonth.SST['mean'], 
                df_keo_avgmonth.SST['std'])#, marker= 's', color='C0',elinewidth=1, linewidth=3)
ax.set_xlabel('Month')#, fontsize=fs)
ax.set_ylabel('SST')#, fontsize=fs)
ax.set_xticks(np.arange(1, 13, step=2))
plt.savefig('keo_avgmonth_sst.png', bbox_inches='tight', dpi=600)
plt.show() # displays plot

Next, we want to calculate solubility-related changes in pCO2,'thermal component' and decompose from the non-thermal component. First, let's look at the monthly pCO2 data.

In [ ]:
# plot monthly averages for pCO2
fig, ax = plt.subplots()
im = ax.errorbar(df_keo_avgmonth.index, df_keo_avgmonth.pCO2_sw['mean'], 
                df_keo_avgmonth.pCO2_sw['std'])#, marker= 's', color='C0',elinewidth=1, linewidth=3)
ax.set_xlabel('Month')#, fontsize=fs)
ax.set_ylabel('seawater pCO2')#, fontsize=fs)
ax.set_xticks(np.arange(1, 13, step=2))
plt.savefig('keo_avgmonth_pCO2sw.png', bbox_inches='tight', dpi=600)
plt.show() # displays plot

In [ ]:
SSTi=df_keo.SST[0]
pCO2_swi=df_keo.pCO2_sw[0]
pCO2_thermal=pCO2_swi+pCO2_swi*.0423*(df_keo.SST-SSTi) # 0.0423 as per Takahashi et al. (1993)


In [ ]:
# plotting pCO2 and pCO2thermal on one plot
fig, ax = plt.subplots()

im = ax.plot(df_keo.index, df_keo.pCO2_sw,label='pCO2')
im2 = ax.plot(df_keo.index, pCO2_thermal,label='thermal')
im3 = ax.plot(df_keo.index, df_keo.pCO2_sw-pCO2_thermal+pCO2_swi,label='nonthermal')

ax.set_ylabel('pCO2') # fontsize=fs)
ax.legend()

plt.savefig('keo_pCO2thermal.png', bbox_inches='tight', dpi=600) # saves plot as image file
plt.show() # displays plot